# Accessing ACLED Data Programmatic Regional Comparison
ACLED API documentation: https://acleddata.com/acleddatanew/wp-content/uploads/dlm_uploads/2019/01/API-User-Guide2020.pdf


#### The notebook is designed to programmatically compare two regoins of ACLED data to each other

# Info for Variables
- start_date and end_date
    - The time range you want to search on.  Formatting is YYYY-MM-DD 
- reg
    - Select one regions of interest
        - 1 - Western Africa
        - 2 - Middle Africa
        - 3 - Eastern Africa
        - 4 - Southern Africa
        - 5 - Northern Africa
        - 7 - Southern Asia
        - 9 - South-Eastern Asia
        - 11 - Middle East
        - 12 - Europe
        - 13 - Caucasus and Central Asia
        - 14 - Central America
        - 15 - South America
        - 16 - Caribbean
   
* Note - Make sure that all variables have quotes around them (example: '12')

# Populate Variables

In [21]:
start_date = '2019-01-01'
end_date = '2019-04-08'
reg = '14'

## Shouldn't have to change anything in the cells below

In [22]:
##Dictionaries to make output data more understandable.  Conversts coded values into text values. 
##Example: value 3 to rebel groups and value 2 to Middle Africa
act_dict = {"1": 'Governments and State Security Services', "2":'Rebel Groups', "3": 'Political Militia', 
            "4": 'Identity Militias', "5": 'Rioters', "6": 'Protestors', "7": 'Civilians', 
            "8": 'External Forces/Other Forces'}
dict_reg = {"1": 'Western Africa', "2":'Middle Africa', "3": 'Eastern Africa', "4": 'Southern Africa', 
            "5": 'Northern Africa', "7": 'Southern Asia', "9": 'South-Eastrn Asia', "11": 'Middle East',
            "12":"Europe","13":"Caucasus and Central Asia","14":"Central America","15":"South America","16":"Caribbean"}


### Importing Packages and Libraries

In [23]:
##import libraries and packages
import requests
import pandas as pd
from pandas.io.json import json_normalize 

### Provides total count by region. This is used to expand the limit of the API query

In [24]:
##API call for regions
URL = "https://api.acleddata.com/region/read?terms=accept"
cnts = requests.get(url = URL)
##Convert to Pandas dataframe
data = cnts.json()
df_reg = json_normalize(data, 'data')
df_reg['count'] = df_reg['event_count'].astype(int)
total = df_reg['count'].sum(axis=0)
##create dictionary for region numeric code and counts of events in each region
dictionary_lim = pd.Series(df_reg.event_count.values, index = df_reg.region).to_dict()
##Get total count of events worldwide 
df_reg['count'] = df_reg['event_count'].astype(int)
total = df_reg['count'].sum(axis=0)
print(total)
##create dictionary for region numeric code and counts of events in each region
dictionary_lim = pd.Series(df_reg.event_count.values, index = df_reg.region).to_dict()
print(dictionary_lim)

758000
{'1': '37238', '2': '29225', '3': '69334', '4': '20951', '5': '50486', '7': '155934', '9': '34661', '11': '195500', '12': '49102', '13': '64367', '14': '21478', '15': '28147', '16': '1577'}


### Query data for Region of interest

In [25]:
###API call - note: limit is determined by dictionary of total counts by region.  
lim = dictionary_lim[reg] 
URL2 = 'https://api.acleddata.com/acled/read?terms=accept'
PARAMS = "&limit="+lim+"&region="+reg+"&event_date="+start_date+"|"+end_date+"&event_date_where=BETWEEN" 
d_req_oi = requests.get(url = URL2, params = PARAMS)
##Convert to Pandas Dataframe
data_event_oi = d_req_oi.json()
df_event_oi = json_normalize(data_event_oi, 'data')

### Count of types of Actor 1s. This is using the inter1 field containing a coded values for the following: 
    Inter Code 1: Governments and State Security Services
    Inter Code 2: Rebel Groups
    Inter Code 3: Political Militia
    Inter Code 4: Identity Militias
    Inter Code 5: Rioters
    Inter Code 6: Protesters
    Inter Code 7: Civilians
    Inter Code 8: External/Other Forces

In [26]:
##dictionary to make columns more understandable by removing the coded values
reg2 = dict_reg[reg]
##Count of actor 1 types for main region of interest
act1_cnt_oi = df_event_oi.groupby('inter1').size().reset_index()
##Rename column from inter1 to actor type and add region name to count column
act1_cnt_oi = act1_cnt_oi.rename(columns={"inter1": "actor_type1", 0: reg2 + "_act1_cnt"})
##replace coded values of actor type to type text
act1_cnt_oi['actor_type1'] = act1_cnt_oi['actor_type1'].map(act_dict)
##Count of actor1 types for main region of interest
act2_cnt_oi = df_event_oi.groupby('inter2').size().reset_index()
##Rename column from inter2 to actor type and add region name to count column
act2_cnt_oi = act2_cnt_oi.rename(columns={"inter2": "actor_type2", 0: reg2 + "_act2_cnt"})
##replace coded values of actor type to type text
act2_cnt_oi['actor_type2'] = act2_cnt_oi['actor_type2'].map(act_dict)

In [27]:
##dictionary of all regions
regs = ["1","2","3","4","5","7","9","11","12","13","14","15","16"]
##removing region of interest because analysis was done in the cell above
regs.remove(reg)
print(regs)

['1', '2', '3', '4', '5', '7', '9', '11', '12', '13', '15', '16']


In [28]:
##Loop through all the regions and get count of actor 1 types
for r in regs: 
    lim = dictionary_lim[r]
    reg2_all = dict_reg[r]
    
    ##query for each region
    URL2 = 'https://api.acleddata.com/acled/read?terms=accept'
    PARAMS = "&limit="+lim+"&region="+r+"&event_date="+start_date+"|"+end_date+"&event_date_where=BETWEEN" 
    d_req = requests.get(url = URL2, params = PARAMS)
    ##save as dataframe
    data_event = d_req.json()
    df_event = json_normalize(data_event, 'data')
    
    ##group by actor type 1 for region
    act1_cnt = df_event.groupby('inter1').size().reset_index()
    ##Rename column from inter1 to actor type and add region name to count column
    act1_cnt = act1_cnt.rename(columns={"inter1": "actor_type1", 0: reg2_all + "_act1_cnt"})
    ##replace coded values of actor type to type text
    act1_cnt['actor_type1'] = act1_cnt['actor_type1'].map(act_dict)
    ##Append counts for each region to dataframe cotaining counts for region of interest 
    act1_cnt_oi = pd.merge(act1_cnt_oi, act1_cnt, on='actor_type1', how='outer')
    
    ##group by actor type 2 for region    
    act2_cnt = df_event.groupby('inter2').size().reset_index()
    ##Rename column from inter1 to actor type and add region name to count column
    act2_cnt = act2_cnt.rename(columns={"inter2": "actor_type2", 0: reg2_all + "_act2_cnt"})
    ##replace coded values of actor type to type text
    act2_cnt['actor_type2'] = act2_cnt['actor_type2'].map(act_dict)
    ##Append counts for each region to dataframe cotaining counts for region of interest 
    act2_cnt_oi = pd.merge(act2_cnt_oi, act2_cnt, on='actor_type2', how='outer')


In [29]:
##show and save counts for actor 1 type
act1_cnt_oi.to_csv(reg2 +'_Actor1_type_cnts.csv')
act1_cnt_oi

,actor_type1,Central America_act1_cnt,Western Africa_act1_cnt,Middle Africa_act1_cnt,Eastern Africa_act1_cnt,Southern Africa_act1_cnt,Northern Africa_act1_cnt,Southern Asia_act1_cnt,South-Eastrn Asia_act1_cnt,Middle East_act1_cnt,Europe_act1_cnt,Caucasus and Central Asia_act1_cnt,South America_act1_cnt,Caribbean_act1_cnt
0,Governments and State Security Services,236.0,193.0,255,328.0,51.0,327.0,357,424,4977,2517.0,2257.0,734,17.0
1,Political Militia,2519.0,342.0,419,467.0,30.0,90.0,529,175,1722,202.0,96.0,1922,189.0
2,Identity Militias,23.0,195.0,37,171.0,5.0,32.0,152,8,55,NaN,NaN,25,NaN
3,Rioters,289.0,181.0,83,106.0,218.0,95.0,1436,57,315,172.0,10.0,473,83.0
4,Protestors,1451.0,378.0,124,121.0,251.0,1709.0,6724,364,1582,2214.0,303.0,2646,62.0
5,Civilians,1.0,NaN,5,NaN,NaN,NaN,2,3,113,1.0,2.0,10,NaN
6,External Forces/Other Forces,2.0,16.0,12,52.0,NaN,11.0,59,1,1887,8.0,699.0,9,NaN
7,Rebel Groups,NaN,298.0,191,304.0,NaN,67.0,206,224,1290,1504.0,1506.0,52,NaN


In [30]:
##show and save counts for actor 2 type
act2_cnt_oi.to_csv(reg2 + '_Actor2_type_cnts.csv')
act2_cnt_oi

,actor_type2,Central America_act2_cnt,Western Africa_act2_cnt,Middle Africa_act2_cnt,Eastern Africa_act2_cnt,Southern Africa_act2_cnt,Northern Africa_act2_cnt,Southern Asia_act2_cnt,South-Eastrn Asia_act2_cnt,Middle East_act2_cnt,Europe_act2_cnt,Caucasus and Central Asia_act2_cnt,South America_act2_cnt,Caribbean_act2_cnt
0,NaN,1524.0,372,154,137,325.0,1568,6613.0,382,5412,3241.0,301.0,2752,82.0
1,Governments and State Security Services,403.0,273,203,314,94.0,351,1063.0,198,2039,1228.0,2913.0,513,58.0
2,Political Militia,357.0,65,125,47,3.0,93,82.0,10,815,51.0,303.0,1392,21.0
3,Identity Militias,13.0,82,24,67,3.0,12,54.0,3,60,NaN,NaN,1,NaN
4,Rioters,20.0,44,10,13,5.0,10,246.0,13,256,19.0,4.0,17,4.0
5,Protestors,8.0,4,1,2,1.0,5,NaN,5,41,14.0,1.0,19,NaN
6,Civilians,2190.0,659,517,758,121.0,215,1219.0,458,1960,240.0,148.0,1106,182.0
7,External Forces/Other Forces,6.0,13,10,73,3.0,9,161.0,2,189,8.0,12.0,13,4.0
8,Rebel Groups,NaN,91,82,138,NaN,68,27.0,185,1169,1817.0,1191.0,58,NaN
